In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
import glob
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.fft import fft, fftfreq
from scipy.signal import firwin, lfilter, butter
# import asrpy
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras import backend as K #转换为张量


2022-06-13 20:04:21.194489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:
2022-06-13 20:04:21.194509: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
with open('labels.pickle', 'rb') as f:
    labels = pickle.load(f)

with open('dataset_Bandpass.pickle', 'rb') as f:
    dataset = pickle.load(f)


把000000的labelindex挑出來

In [4]:
c_0 = []
c = []
for i in range(0,len(labels)):
    if labels[i].sum() == 0:
        c_0.append(i)
    else:
        c.append(i)

把000000的label跟data分開和其他label跟data分開

In [6]:
label_0 = []
label_1 = []
data_0 = []
data_1 = []
for i in c_0:
    label_0.append(np.array(labels[i]))
    data_0.append(dataset[i])
for i in c:
    label_1.append(np.array(labels[i]))
    data_1.append(dataset[i])

減少屬於000000label 的data數量，使數量跟其他label接近

In [8]:
datax, data_00 = train_test_split(data_0,test_size=0.0035,train_size=1-0.0035)
datax = []
len(data_00)

349

In [9]:
data = data_00+data_1
label = label_0[:len(data_00)]+label_1

In [11]:
data = np.array(data)
label = np.array(label)

打亂訓練數據

In [12]:
datax, data, labelx, label = train_test_split(data, label,test_size=0.99,train_size=0.01)

In [14]:
X_data = data.astype('float64')
Y_data = label.astype('int64')
# X_train = np.array(dataset_fft)
X_data = X_data.reshape(len(X_data), X_data[0].shape[0], X_data[0].shape[1], 1)
# Y_train = np.array(labels)

打亂訓練數據

In [21]:

def new_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, padding = "same", kernel_size = (X_data.shape[1],3), activation = "relu", input_shape = (X_data.shape[1], X_data.shape[2], 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 64, padding = "same", kernel_size = (1,X_data.shape[2]), activation = "relu", input_shape = (X_data.shape[1], X_data.shape[2], 1)))
    model.add(BatchNormalization())
    # model.add(Conv2D(filters = 64, padding = "same", kernel_size = (3,3), activation = "relu", input_shape = (X_data.shape[1], X_data.shape[2], 1)))
    # model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dense(64, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dense(6, activation = "sigmoid"))

    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])
    
    return model

In [22]:
data_fft_model = new_model()
data_fft_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 100, 32, 32)       9632      
                                                                 
 batch_normalization_8 (Batc  (None, 100, 32, 32)      128       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 100, 32, 64)       65600     
                                                                 
 batch_normalization_9 (Batc  (None, 100, 32, 64)      256       
 hNormalization)                                                 
                                                                 
 flatten_2 (Flatten)         (None, 204800)            0         
                                                                 
 dense_6 (Dense)             (None, 256)              

In [23]:
train_history = data_fft_model.fit(X_data, Y_data, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
31/31 [==============================] - 20s 616ms/step - loss: 0.7377 - accuracy: 0.2032 - val_loss: 0.6276 - val_accuracy: 0.1463
Epoch 2/10
31/31 [==============================] - 19s 624ms/step - loss: 0.6186 - accuracy: 0.3432 - val_loss: 0.5793 - val_accuracy: 0.2358
Epoch 3/10
31/31 [==============================] - 19s 625ms/step - loss: 0.5468 - accuracy: 0.4695 - val_loss: 0.5450 - val_accuracy: 0.2337
Epoch 4/10
31/31 [==============================] - 19s 626ms/step - loss: 0.4600 - accuracy: 0.5916 - val_loss: 0.4909 - val_accuracy: 0.2358
Epoch 5/10
31/31 [==============================] - 19s 625ms/step - loss: 0.3703 - accuracy: 0.6787 - val_loss: 0.4367 - val_accuracy: 0.3028
Epoch 6/10
31/31 [==============================] - 19s 626ms/step - loss: 0.2822 - accuracy: 0.7581 - val_loss: 0.4091 - val_accuracy: 0.4390
Epoch 7/10
31/31 [==============================] - 19s 630ms/step - loss: 0.2066 - accuracy: 0.8086 - val_loss: 0.3929 - val_accuracy: 0.4797

打亂訓練數據

In [24]:
with open('test_labels.pickle', 'rb') as f:
    tlabels = pickle.load(f)
    
with open('test_Bandpass.pickle', 'rb') as f:
    tdataset = pickle.load(f)

In [25]:
c_0 = []
c = []
for i in range(0,len(tlabels)):
    if tlabels[i].sum() == 0:
        c_0.append(i)
    else:
        c.append(i)

In [26]:
label_0 = []
label_1 = []
data_0 = []
data_1 = []
for i in c_0:
    label_0.append(np.array(tlabels[i]))
    data_0.append(np.array(tdataset[i]))
for i in c:
    label_1.append(np.array(tlabels[i]))
    data_1.append(np.array(tdataset[i]))
    
label_1 = np.array(label_1)
data_1 = np.array(data_1)
label_1 = label_1.astype('float64')
data_1 = data_1.astype('int64')

In [27]:
t_labels = []
t_dataset = []
for i in range(len(tlabels)):
    t_labels.append(np.array(tlabels[i]))
    t_dataset.append(np.array(tdataset[i]))
    
tdataset = np.array(t_dataset)
tlabels = np.array(t_labels)
t_labels = []
t_dataset = []

In [28]:
tdataset = tdataset.astype('float64')
tlabels = tlabels.astype('int64')

In [29]:
scores=data_fft_model.evaluate(tdataset,tlabels)
print()
print('accuracy',scores[1])

556/556 [==============================] - 34s 62ms/step - loss: 0.1902 - accuracy: 0.5705

accuracy 0.5704962015151978


打亂訓練數據

In [30]:
scores=data_fft_model.evaluate(data_1,label_1)
print()
print('accuracy',scores[1])

24/24 [==============================] - 2s 61ms/step - loss: 0.4985 - accuracy: 0.3034

accuracy 0.30343008041381836


In [31]:
# 模型結構存檔
from keras.models import model_from_json
json_string = data_fft_model.to_json()
with open("Bandpass.config", "w") as text_file:
    text_file.write(json_string)

    
# 模型訓練結果存檔
data_fft_model.save_weights("Bandpass.weight")